In [105]:
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

In [106]:
def get_publisher_download_url(crossref_resp):
    # search for 'text-mining' in the intended-application field
    text_mining = []
    for link in crossref_response.json()['link']:
        if re.search(r'text-mining',link['intended-application']):
            text_mining.append(re.search('text-mining', link['intended-application']))
        else:
            text_mining.append(None)
            
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(text_mining) != None)[0].tolist()
    # wiley_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [108]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + '/' + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [109]:
with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/wiley/wiley_config.json') as config_file:
    wiley_token = json.load(config_file)['api_key']

In [110]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]
len(pdfs)

13091

In [111]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
wiley_oa_articles = articles[(articles['publisher'] == 'Wiley') & (articles['is_oa'] == True)].drop_duplicates().reset_index(drop=True)

In [112]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
wiley_oa_articles['doi_suffix'] = wiley_oa_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())

In [113]:
full_texts_to_download = [x for x in wiley_oa_articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), wiley_oa_articles, how='inner', on='doi_suffix')
merge.shape

(297, 50)

In [114]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_oa'

In [124]:
# n = random.randint(0, merge.shape[0])
# row = merge.loc[n,:]
# doi = row['doi']
# doi

'10.1002/clc.4960101111'

In [125]:
# doi = row['doi']
# out = {'doi': doi}
# doi_url = 'https://doi.org/' + str(doi)
# crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
# try:
#     crossref_response = requests.get(
#                     url = doi_url,
#                     headers=crossref_headers,
#                     allow_redirects=True,
#                     verify=True)
#     out.update({'CR_status_code': crossref_response.status_code})

#     if crossref_response.status_code == 200:
#         full_text_url = get_publisher_download_url(crossref_response)
#         publisher_headers = {
#             'User-Agent': user_agent,
#             'Wiley-TDM-Client-Token': wiley_token,
#             # 'CR-TDM-Rate-Limit': '1000',
#             # 'CR-TDM-Rate-Limit-Remaining': '76',
#             # 'CR-TDM-Rate-Limit-Reset': '1378072800'
#         }
#         publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
#         out.update({'publisher_status_code': publisher_response.status_code})
#         print(publisher_response.status_code)
# except Exception as e:
#     print(e)
#     out.update({'error': e})


200


In [126]:
log = []

In [ ]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})

        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response)
            publisher_headers = {
                'User-Agent': user_agent,
                'Wiley-TDM-Client-Token': wiley_token,
                # 'CR-TDM-Rate-Limit': '1000',
                # 'CR-TDM-Rate-Limit-Remaining': '76',
                # 'CR-TDM-Rate-Limit-Reset': '1378072800'
            }
            publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
            out.update({'publisher_status_code': publisher_response.status_code})
            if publisher_response.status_code == 200:
                download_pdf(doi = doi, dest_folder=dest_folder, content=publisher_response.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    time.sleep(0.35)

    log.append(out)

 24%|██▍       | 72/297 [01:05<03:32,  1.06it/s]

list index out of range


 25%|██▌       | 75/297 [01:13<08:05,  2.19s/it]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1113%2Fjphysiol.2003.042747 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8154aa0e20>: Failed to establish a new connection: [Errno 61] Connection refused'))


 26%|██▌       | 76/297 [01:14<06:45,  1.83s/it]

...

 26%|██▋       | 78/297 [01:16<04:48,  1.32s/it]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1113%2Fjphysiol.2002.029108 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8151e2a340>: Failed to establish a new connection: [Errno 61] Connection refused'))


 27%|██▋       | 79/297 [01:17<04:13,  1.16s/it]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1111%2Fj.1469-7793.2001.00283.x (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8151e2af70>: Failed to establish a new connection: [Errno 61] Connection refused'))


 31%|███       | 92/297 [02:21<17:20,  5.08s/it]

In [ ]:
log_df = pd.DataFrame(log)
log_df['publisher_status_code'].value_counts()

In [ ]:
log_df[log_df['publisher_status_code'] == 200].reset_index(drop=True)

In [48]:
non200 = log_df[log_df['publisher_status_code'] != 200].reset_index(drop=True)
non200.shape

(379, 4)

In [49]:
wiley_errors = pd.merge(non200, merge, how='inner', on='doi')
wiley_errors.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_oa_status_codes.csv',
                    index=False)

In [50]:
wiley_errors.shape

(379, 53)